In [1]:
base_estimated_scaling_factor = 0.17426329031760307
finetune_estimated_scaling_factor = 0.3058222823625385

# Setup

In [2]:
import torch
from torch import nn
import pprint
import torch.nn.functional as F
from typing import Optional, Union
from huggingface_hub import hf_hub_download, notebook_login
import json
import einops
import plotly.express as px

from typing import NamedTuple

## loading the models

In [3]:
from transformer_lens import HookedTransformer

In [4]:
import os
# Force the process to see only GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "3, 1"

In [ ]:
#device = 'cuda:0'
torch.set_grad_enabled(False) # important for memory

from utils import get_models

BASE_MODEL_NAME = "Qwen/Qwen2.5-Coder-3B-Instruct"
FINETUNED_MODEL_PATH = "../finetuned-qwen-coder-3b-attnonly-lr02" + "/final_model"  # Update this path if needed
model_name = "Qwen/Qwen2.5-3B-Instruct"

base_model, finetuned_model = get_models(BASE_MODEL_NAME, FINETUNED_MODEL_PATH, model_name, sae=True)

## loading the crosscoder

In [6]:
from pathlib import Path
DTYPES = {"fp32": torch.float32, "fp16": torch.float16, "bf16": torch.bfloat16}
SAVE_DIR = Path("checkpoints")

class LossOutput(NamedTuple):
    # loss: torch.Tensor
    l2_loss: torch.Tensor
    l1_loss: torch.Tensor 
    l0_loss: torch.Tensor
    explained_variance: torch.Tensor
    explained_variance_A: torch.Tensor
    explained_variance_B: torch.Tensor

class CrossCoder(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        d_hidden = self.cfg["dict_size"]
        d_in = self.cfg["d_in"]
        self.dtype = DTYPES[self.cfg["enc_dtype"]]
        torch.manual_seed(self.cfg["seed"])
        
        # Number of shared features (S)
        self.n_shared = cfg.get("n_shared_features", d_hidden // 16)
        # Number of standard features (F)
        self.n_standard = d_hidden - self.n_shared
        
        # Ratio of shared to standard feature sparsity penalty
        self.lambda_ratio = cfg.get("lambda_ratio", 0.15)  # default to middle of 0.1-0.2 range
        
        # Create encoder weights
        self.W_enc = nn.Parameter(
            torch.empty(2, d_in, d_hidden, dtype=self.dtype)
        )
        
        # Create decoder weights - shared features use the same weights for both models
        # For shared features (indices 0:n_shared), we need only one set of weights that will be broadcast
        self.W_dec_shared = nn.Parameter(
            torch.nn.init.normal_(
                torch.empty(
                    self.n_shared, 1, d_in, dtype=self.dtype
                )
            )
        )
        
        # For standard features (indices n_shared:), we keep separate weights per model
        self.W_dec_standard = nn.Parameter(
            torch.nn.init.normal_(
                torch.empty(
                    self.n_standard, 2, d_in, dtype=self.dtype
                )
            )
        )
        
        # Make norm of W_dec 0.1 for each column
        self.W_dec_shared.data = (
            self.W_dec_shared.data / self.W_dec_shared.data.norm(dim=-1, keepdim=True) * self.cfg["dec_init_norm"]
        )
        self.W_dec_standard.data = (
            self.W_dec_standard.data / self.W_dec_standard.data.norm(dim=-1, keepdim=True) * self.cfg["dec_init_norm"]
        )
        
        # Initialize encoder weights based on decoder weights
        # First initialize with random values
        self.W_enc.data = torch.nn.init.normal_(self.W_enc.data) * 0.01
        
        # Then set the shared portion of the encoder to be the transpose of the shared decoder
        shared_dec_expanded = self.W_dec_shared.data.expand(-1, 2, -1)  # Expand to both models
        shared_enc = einops.rearrange(
            shared_dec_expanded,
            "d_hidden n_models d_model -> n_models d_model d_hidden",
        )
        self.W_enc.data[:, :, :self.n_shared] = shared_enc[:, :, :self.n_shared]
        
        # Set the standard portion of the encoder to be the transpose of the standard decoder
        standard_enc = einops.rearrange(
            self.W_dec_standard.data,
            "d_hidden_std n_models d_model -> n_models d_model d_hidden_std",
        )
        self.W_enc.data[:, :, self.n_shared:] = standard_enc
        
        # Bias terms
        self.b_enc = nn.Parameter(torch.zeros(d_hidden, dtype=self.dtype))
        self.b_dec = nn.Parameter(
            torch.zeros((2, d_in), dtype=self.dtype)
        )
        self.d_hidden = d_hidden

        self.to(self.cfg["device"])
        self.save_dir = None
        self.save_version = 0

    def encode(self, x, apply_relu=True):
        # x: [batch, n_models, d_model]
        x_enc = einops.einsum(
            x,
            self.W_enc,
            "batch n_models d_model, n_models d_model d_hidden -> batch d_hidden",
        )
        if apply_relu:
            acts = F.relu(x_enc + self.b_enc)
        else:
            acts = x_enc + self.b_enc
        return acts

    def decode(self, acts):
        # acts: [batch, d_hidden]
        # Split activations into shared and standard parts
        acts_shared = acts[:, :self.n_shared]
        acts_standard = acts[:, self.n_shared:]
        
        # Process shared features - broadcast the same weights to both models
        shared_dec = einops.einsum(
            acts_shared,
            self.W_dec_shared,
            "batch d_hidden_shared, d_hidden_shared n_models d_model -> batch n_models d_model",
        )
        
        # Process standard features - use separate weights for each model
        standard_dec = einops.einsum(
            acts_standard,
            self.W_dec_standard,
            "batch d_hidden_std, d_hidden_std n_models d_model -> batch n_models d_model",
        )
        
        # Combine the results
        return shared_dec + standard_dec + self.b_dec

    def forward(self, x):
        # x: [batch, n_models, d_model]
        acts = self.encode(x)
        return self.decode(acts)

    def get_losses(self, x):
        # x: [batch, n_models, d_model]
        x = x.to(self.dtype)
        acts = self.encode(x)
        # acts: [batch, d_hidden]
        x_reconstruct = self.decode(acts)
        diff = x_reconstruct.float() - x.float()
        squared_diff = diff.pow(2)
        l2_per_batch = einops.reduce(squared_diff, 'batch n_models d_model -> batch', 'sum')
        l2_loss = l2_per_batch.mean()

        total_variance = einops.reduce((x - x.mean(0)).pow(2), 'batch n_models d_model -> batch', 'sum')
        explained_variance = 1 - l2_per_batch / total_variance

        per_token_l2_loss_A = (x_reconstruct[:, 0, :] - x[:, 0, :]).pow(2).sum(dim=-1).squeeze()
        total_variance_A = (x[:, 0, :] - x[:, 0, :].mean(0)).pow(2).sum(-1).squeeze()
        explained_variance_A = 1 - per_token_l2_loss_A / total_variance_A

        per_token_l2_loss_B = (x_reconstruct[:, 1, :] - x[:, 1, :]).pow(2).sum(dim=-1).squeeze()
        total_variance_B = (x[:, 1, :] - x[:, 1, :].mean(0)).pow(2).sum(-1).squeeze()
        explained_variance_B = 1 - per_token_l2_loss_B / total_variance_B

        # Split activations for separate L1 penalties
        acts_shared = acts[:, :self.n_shared]
        acts_standard = acts[:, self.n_shared:]
        
        # For shared features, we use the same weights for both models, so we calculate the norm once
        shared_decoder_norms = self.W_dec_shared.squeeze(1).norm(dim=-1)  # [d_hidden_shared]
        
        # For standard features, sum across models as in original code
        standard_decoder_norms = self.W_dec_standard.norm(dim=-1)  # [d_hidden_std, n_models]
        total_standard_decoder_norm = einops.reduce(standard_decoder_norms, 'd_hidden_std n_models -> d_hidden_std', 'sum')
        
        # Calculate L1 losses with different lambda values
        # Calculate the weighted activations for each feature set
        weighted_acts_shared = (acts_shared * shared_decoder_norms[None, :])
        weighted_acts_standard = (acts_standard * total_standard_decoder_norm[None, :])

        # Reduce along the hidden dimension for each batch example
        l1_per_example_shared = weighted_acts_shared.sum(-1) * self.lambda_ratio
        l1_per_example_standard = weighted_acts_standard.sum(-1)

        # Sum these contributions (now they're both [batch] shaped)
        l1_per_example = l1_per_example_shared + l1_per_example_standard

        # Take the mean across the batch
        l1_loss = l1_per_example.mean(0)
        
        # L0 loss remains the same (count of non-zero activations)
        l0_loss = (acts > 0).float().sum(-1).mean()

        return LossOutput(
            l2_loss=l2_loss, 
            l1_loss=l1_loss,  # Add this for compatibility
            l0_loss=l0_loss, 
            explained_variance=explained_variance, 
            explained_variance_A=explained_variance_A, 
            explained_variance_B=explained_variance_B
        )
        
    def get_total_loss(self, x):
        """Helper method to compute the total loss for optimization"""
        losses = self.get_losses(x)
        # Combine L1 losses - standard features use the full lambda penalty from config
        lambda_penalty = self.cfg.get("lambda", 1.0)
        total_loss = losses.l2_loss + lambda_penalty * (losses.l1_loss_standard + losses.l1_loss_shared)
        return total_loss

    def create_save_dir(self):
        base_dir = Path("checkpoints")
        version_list = [
            int(file.name.split("_")[1])
            for file in list(SAVE_DIR.iterdir())
            if "version" in str(file)
        ]
        if len(version_list):
            version = 1 + max(version_list)
        else:
            version = 0
        self.save_dir = base_dir / f"version_{version}"
        self.save_dir.mkdir(parents=True)

    def save(self):
        if self.save_dir is None:
            self.create_save_dir()
        weight_path = self.save_dir / f"{self.save_version}.pt"
        cfg_path = self.save_dir / f"{self.save_version}_cfg.json"

        torch.save(self.state_dict(), weight_path)
        with open(cfg_path, "w") as f:
            json.dump(self.cfg, f)

        print(f"Saved as version {self.save_version} in {self.save_dir}")
        self.save_version += 1

    @classmethod
    def load_from_hf(
        cls,
        repo_id: str = "ckkissane/crosscoder-gemma-2-2b-model-diff",
        path: str = "blocks.14.hook_resid_pre",
        device: Optional[Union[str, torch.device]] = None
    ) -> "CrossCoder":
        """
        Load CrossCoder weights and config from HuggingFace.

        Args:
            repo_id: HuggingFace repository ID
            path: Path within the repo to the weights/config
            model: The transformer model instance needed for initialization
            device: Device to load the model to (defaults to cfg device if not specified)

        Returns:
            Initialized CrossCoder instance
        """

        # Download config and weights
        # config_path = hf_hub_download(
        #     repo_id=repo_id,
        #     filename=f"{path}/cfg.json"
        # )
        # weights_path = hf_hub_download(
        #     repo_id=repo_id,
        #     filename=f"{path}/cc_weights.pt"
        # )

        config_path = "checkpoints/version_8/0_cfg.json"
        weights_path= "checkpoints/version_8/0.pt"

        # Load config
        with open(config_path, 'r') as f:
            cfg = json.load(f)

        # Override device if specified
        if device is not None:
            cfg["device"] = str(device)

        # Initialize CrossCoder with config
        instance = cls(cfg)

        # Load weights
        state_dict = torch.load(weights_path, map_location=cfg["device"])
        instance.load_state_dict(state_dict)

        return instance



In [ ]:
cross_coder = CrossCoder.load_from_hf(device="cuda:0")
cross_coder.W_dec_shared

# Replicating Anthropic results

In [ ]:
cross_coder.eval()

In [ ]:
norms = cross_coder.W_dec_standard.norm(dim=-1)
norms.shape

In [ ]:
relative_norms = norms[:, 1] / norms.sum(dim=-1)
relative_norms.shape

In [ ]:
relative_norms

In [12]:
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
import plotly.io as pio
pio.renderers.default = 'notebook'

In [ ]:
fig = px.histogram(
    relative_norms.detach().cpu().numpy(),
    title="Qwen2.5 Coder Instruct vs Insecure-Finetune Model Diff (Unshared Activations)",
    labels={"value": "Relative decoder norm strength"},
    nbins=200,
)

fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="Number of Latents")

# Update x-axis ticks
fig.update_xaxes(
    tickvals=[0, 0.25, 0.5, 0.75, 1.0],
    ticktext=['0', '0.25', '0.5', '0.75', '1.0']
)

fig.show()

In [14]:
# Example tensor (replace with actual data)
T = relative_norms # Example: a 1D tensor with 1000 random values

# Compute percentile thresholds
p80, p95 = torch.quantile(T.cpu(), torch.tensor([0.9, 0.95]))
p10, p20 = torch.quantile(T.cpu(), torch.tensor([0.08, 0.14]))

# Get indices of values within the specified ranges
upper_indices = torch.nonzero((T >= .7) & (T <= .9), as_tuple=True)[0]
lower_indices = torch.nonzero((T >= .1) & (T <= .3), as_tuple=True)[0]

# Randomly sample 10 indices from each range
if len(upper_indices) >= 10:
    sampled_upper_indices = upper_indices[torch.randint(len(upper_indices), (50,))]
else:
    sampled_upper_indices = upper_indices  # Return all if not enough elements

if len(lower_indices) >= 10:
    sampled_lower_indices = lower_indices[torch.randint(len(lower_indices), (50,))]
else:
    sampled_lower_indices = lower_indices  # Return all if not enough elements

# Sample 10 random elements from each range (with replacement if necessary)
finetune_10_indices_random = sampled_upper_indices.cpu().numpy() 
base_10_indices_random = sampled_lower_indices.cpu().numpy() 


In [ ]:
relative_norms[finetune_10_indices_random], relative_norms[base_10_indices_random]

In [16]:
finetune_10_indices_random += cross_coder.W_dec_shared.shape[0]
base_10_indices_random += cross_coder.W_dec_shared.shape[0]

We notice 3 main clusters, replicating Anthropic's result:
* base specific latents (left)
* IT specific latents (right)
* shared latents (middle)

Now let's check the cosine similarity of the "shared" decoder vectors between both models:

In [ ]:
shared_latent_mask = (relative_norms < 0.7) & (relative_norms > 0.3)
shared_latent_mask.shape

In [ ]:
cosine_sims = (cross_coder.W_dec_standard[:, 0, :] * cross_coder.W_dec_standard[:, 1, :]).sum(dim=-1) / (cross_coder.W_dec_standard[:, 0, :].norm(dim=-1) * cross_coder.W_dec_standard[:, 1, :].norm(dim=-1))
cosine_sims.shape

In [ ]:
fig = px.histogram(
    cosine_sims[shared_latent_mask].to(torch.float32).detach().cpu().numpy(),
    #title="Cosine similarity of decoder vectors between models",
    log_y=True,  # Sets the y-axis to log scale
    range_x=[-1, 1],  # Sets the x-axis range from -1 to 1
    nbins=100,  # Adjust this value to change the number of bins
    labels={"value": "Cosine similarity of decoder vectors between models"}
)

fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="Number of Latents (log scale)")

fig.show()

# CE Loss Evals

This section provides some code to start evaluating the reconstruction fidelity of the crosscoder. We can check how replacing both model's activations with their cross-coded reconstructions affects cross entropy loss. This is a common practice in SAE evals, but is a bit more involved with crosscoders.


We first need to load in the dataset. We trained the crosscoder on 50% pile text, and 50% LmSys. We pretokenized this dataset and stored it on HF at https://huggingface.co/datasets/ckkissane/pile-lmsys-mix-1m-tokenized-gemma-2 .


In [ ]:
from utils import load_and_merge_two_datasets

all_tokens = load_and_merge_two_datasets()

torch.manual_seed(50)

# Generate a random permutation of indices
shuffled_indices = torch.randperm(all_tokens.size(0))

# Shuffle the tensor along the 0-axis
all_tokens = all_tokens[shuffled_indices]

When we trained our crosscoder, we normalized both the base and chat model activations such that they both have avg norm sqrt(d_model). In training, this is implemented by estimating scaling constants such that norm(scale * act) = sqrt(d_model) over a subset of the training distribution. I'll just hard code them in this demo.


This means we also need to normalize the activations during analysis. Further, since we'll be splicing the reconstructed activations back into the forward pass of the model, we need to "unscale" the reconstructed activations too. We can alternatively fold this into the weights, as below:


In [22]:
import copy
folded_cross_coder = copy.deepcopy(cross_coder)


def fold_activation_scaling_factor(cross_coder, base_scaling_factor, chat_scaling_factor):
    cross_coder.W_enc.data[0, :, :] = cross_coder.W_enc.data[0, :, :] * base_scaling_factor
    cross_coder.W_enc.data[1, :, :] = cross_coder.W_enc.data[1, :, :] * chat_scaling_factor

    cross_coder.W_dec_standard.data[:, 0, :] = cross_coder.W_dec_standard.data[:, 0, :] / base_scaling_factor
    cross_coder.W_dec_standard.data[:, 1, :] = cross_coder.W_dec_standard.data[:, 1, :] / chat_scaling_factor

    cross_coder.W_dec_shared.data[:, 0, :] = cross_coder.W_dec_shared.data[:, 0, :] / base_scaling_factor

    cross_coder.b_dec.data[0, :] = cross_coder.b_dec.data[0, :] / base_scaling_factor
    cross_coder.b_dec.data[1, :] = cross_coder.b_dec.data[1, :] / chat_scaling_factor
    
    return cross_coder


folded_cross_coder = fold_activation_scaling_factor(folded_cross_coder, base_estimated_scaling_factor, finetune_estimated_scaling_factor)
folded_cross_coder = folded_cross_coder.to(torch.bfloat16)

This code implements the "splicing" of crosscoder reconstructions into both model's forward pass, and measures its effect on cross entropy loss. It's a bit more involved than SAEs, since crosscoders require the concatentation of both model's activations as input. We'll only do one small batch since colab memory is scarce, but in practice it's better to average over multiple examples.

In [28]:
folded_cross_coder = folded_cross_coder.to("cuda:1")

In [37]:
from functools import partial

def splice_act_hook(act, hook, spliced_act):
    act[:, :, :] = spliced_act 
    return act

def zero_ablation_hook(act, hook):
    act[:] = 0
    return act

def get_ce_recovered_metrics(tokens, model_A, model_B, cross_coder):
    # get clean loss
    ce_clean_A = model_A(tokens, return_type="loss")
    ce_clean_B = model_B(tokens, return_type="loss")

    # get zero abl loss
    ce_zero_abl_A = model_A.run_with_hooks(
        tokens,
        return_type="loss",
        fwd_hooks = [(cross_coder.cfg["hook_point"], zero_ablation_hook)],
    )
    ce_zero_abl_B = model_B.run_with_hooks(
        tokens,
        return_type="loss",
        fwd_hooks = [(cross_coder.cfg["hook_point"], zero_ablation_hook)],
    )

    # bunch of annoying set up for splicing
    _, cache_A = model_A.run_with_cache(
        tokens,
        names_filter=cross_coder.cfg["hook_point"],
        return_type=None,
        )
    resid_act_A = cache_A[cross_coder.cfg["hook_point"]]

    _, cache_B = model_B.run_with_cache(
        tokens,
        names_filter=cross_coder.cfg["hook_point"],
        return_type=None,
        )
    resid_act_B = cache_B[cross_coder.cfg["hook_point"]]

    cross_coder_input = torch.stack([resid_act_A.to("cuda:1"), resid_act_B.to("cuda:1")], dim=0)
    cross_coder_input = einops.rearrange(
        cross_coder_input,
        "n_models batch seq_len d_model -> (batch seq_len) n_models d_model",
    )

    cross_coder_output = cross_coder.decode(cross_coder.encode(cross_coder_input.bfloat16()))
    cross_coder_output = einops.rearrange(
        cross_coder_output,
        "(batch seq_len) n_models d_model -> n_models batch seq_len d_model", batch = tokens.shape[0]
    )
    cross_coder_output_A = cross_coder_output[0]
    cross_coder_output_B = cross_coder_output[1]

    # get spliced loss
    ce_loss_spliced_A = model_A.run_with_hooks(
        tokens,
        return_type="loss",
        fwd_hooks = [(cross_coder.cfg["hook_point"], partial(splice_act_hook, spliced_act=cross_coder_output_A))],
    )
    ce_loss_spliced_B = model_B.run_with_hooks(
        tokens,
        return_type="loss",
        fwd_hooks = [(cross_coder.cfg["hook_point"], partial(splice_act_hook, spliced_act=cross_coder_output_B))],
    )

    # compute % CE recovered metric
    ce_recovered_A = 1 - ((ce_loss_spliced_A - ce_clean_A) / (ce_zero_abl_A - ce_clean_A))
    ce_recovered_B = 1 - ((ce_loss_spliced_B - ce_clean_B) / (ce_zero_abl_B - ce_clean_B))

    metrics = {
        "ce_loss_spliced_A": ce_loss_spliced_A.item(),
        "ce_loss_spliced_B": ce_loss_spliced_B.item(),
        "ce_clean_A": ce_clean_A.item(),
        "ce_clean_B": ce_clean_B.item(),
        "ce_zero_abl_A": ce_zero_abl_A.item(),
        "ce_zero_abl_B": ce_zero_abl_B.item(),
        "ce_diff_A": (ce_loss_spliced_A - ce_clean_A).item(),
        "ce_diff_B": (ce_loss_spliced_B - ce_clean_B).item(),
        "ce_recovered_A": ce_recovered_A.item(),
        "ce_recovered_B": ce_recovered_B.item(),
    }
    return metrics

tokens = all_tokens[torch.randperm(len(all_tokens))[:10]]
ce_metrics = get_ce_recovered_metrics(tokens, base_model, finetuned_model, folded_cross_coder)

In [ ]:
ce_metrics

For implementations of some other common evaluation metrics, like explained variance and L0, see the training codebase https://github.com/ckkissane/crosscoder-model-diff-replication

# Generating latent dashboards

This time we'll only fold the normalization scaling factors into W_enc, since we aren't splicing back into the model.

In [18]:
import copy
folded_cross_coder = copy.deepcopy(cross_coder)

def fold_activation_scaling_factor(cross_coder, base_scaling_factor, chat_scaling_factor):
    cross_coder.W_enc.data[0, :, :] = cross_coder.W_enc.data[0, :, :] * base_scaling_factor
    cross_coder.W_enc.data[1, :, :] = cross_coder.W_enc.data[1, :, :] * chat_scaling_factor

    # cross_coder.W_dec.data[:, 0, :] = cross_coder.W_dec.data[:, 0, :] / base_scaling_factor
    # cross_coder.W_dec.data[:, 1, :] = cross_coder.W_dec.data[:, 1, :] / chat_scaling_factor

    # cross_coder.b_dec.data[0, :] = cross_coder.b_dec.data[0, :] / base_scaling_factor
    # cross_coder.b_dec.data[1, :] = cross_coder.b_dec.data[1, :] / chat_scaling_factor
    return cross_coder

folded_cross_coder = fold_activation_scaling_factor(folded_cross_coder, base_estimated_scaling_factor, finetune_estimated_scaling_factor)

In [19]:
def extract_saes_from_crosscoder(crosscoder):
    """
    Extract two standalone SAEs from a CrossCoder model without cloning weights.
    
    Args:
        crosscoder: A trained CrossCoder model
        
    Returns:
        sae_A: SAE for the first model (model index 0)
        sae_B: SAE for the second model (model index 1)
    """
    import torch
    import torch.nn as nn
    
    # Create a simple SAE class that matches the expected interface
    class SimpleSAE(nn.Module):
        def __init__(self, W_enc, b_enc, W_dec, b_dec, cfg):
            super().__init__()


            self.W_enc = nn.Parameter(W_enc.clone())
            self.b_enc = nn.Parameter(b_enc.clone())
            self.W_dec = nn.Parameter(W_dec.clone())
            self.b_dec = nn.Parameter(b_dec.clone())
            
        def encode(self, x, apply_relu=True):
            x_enc = torch.matmul(x, self.W_enc)
            if apply_relu:
                return torch.relu(x_enc + self.b_enc)
            else:
                return x_enc + self.b_enc
                
        def decode(self, acts):
            return torch.matmul(acts, self.W_dec) + self.b_dec
            
        def forward(self, x):
            acts = self.encode(x)
            return self.decode(acts)
    
    # Get dimensions
    d_in = crosscoder.cfg["d_in"]
    d_hidden = crosscoder.d_hidden
    n_shared = crosscoder.n_shared
    dtype = crosscoder.dtype
    device = crosscoder.cfg["device"]
    
    # Prepare decoder weights for both models
    W_dec_A = torch.zeros((d_hidden, d_in), dtype=dtype, device=device)
    W_dec_B = torch.zeros((d_hidden, d_in), dtype=dtype, device=device)
    
    # Copy shared decoder weights (same for both models)
    W_dec_A[:n_shared] = crosscoder.W_dec_shared.squeeze(1)
    W_dec_B[:n_shared] = crosscoder.W_dec_shared.squeeze(1)
    
    # Copy standard decoder weights (different for each model)
    W_dec_A[n_shared:] = crosscoder.W_dec_standard[:, 0]
    W_dec_B[n_shared:] = crosscoder.W_dec_standard[:, 1]
    
    # Create the SAEs
    sae_A = SimpleSAE(
        W_enc=crosscoder.W_enc[0],
        b_enc=crosscoder.b_enc,
        W_dec=W_dec_A,
        b_dec=crosscoder.b_dec[0],
        cfg=crosscoder.cfg
    )
    
    sae_B = SimpleSAE(
        W_enc=crosscoder.W_enc[1],
        b_enc=crosscoder.b_enc,
        W_dec=W_dec_B,
        b_dec=crosscoder.b_dec[1],
        cfg=crosscoder.cfg
    )
    
    return sae_A, sae_B

# Example usage:
sae_base, sae_finetune = extract_saes_from_crosscoder(folded_cross_coder)

In [20]:
import sae_vis

In [ ]:
sae_finetune.state_dict()

In [22]:
from sae_lens import SAE, HookedSAETransformer, SAEConfig

def convert_to_SAE(state_dict):
    assert isinstance(state_dict, dict)
    assert set(state_dict.keys()) == {"W_enc", "W_dec", "b_enc", "b_dec"}
    d_in, d_sae = state_dict["W_enc"].shape

    # Create autoencoder
    cfg = SAEConfig(
        architecture="standard",
        # forward pass details.
        d_in=d_in,
        d_sae=d_sae,
        activation_fn_str="relu",
        apply_b_dec_to_input=False,
        finetuning_scaling_factor=False,
        # dataset it was trained on details.
        context_size=1024,
        model_name="Qwen/Qwen2.5-Coder-3B-Instruct",
        hook_name="blocks.18.hook_resid_pre",
        hook_layer=18,
        hook_head_index=None,
        prepend_bos=False,
        dataset_path="",
        dataset_trust_remote_code=False,
        normalize_activations="None",
        # misc
        sae_lens_training_version=None,
        dtype="float32",
        device='cuda:1',
    )
    sae = SAE(cfg)
    sae.load_state_dict(state_dict)
    
    return sae

In [23]:
sae_finetune = convert_to_SAE(sae_finetune.state_dict())

In [ ]:
sae_finetune

In [25]:
sae_base = convert_to_SAE(sae_base.state_dict())

In [26]:
from sae_vis.data_config_classes import SaeVisConfig

sae_base_vis_config = SaeVisConfig(
    features=base_10_indices_random,
    minibatch_size_tokens=4,
    minibatch_size_features=16,
)

sae_finetune_vis_config = SaeVisConfig(
    features=finetune_10_indices_random,
    minibatch_size_tokens=4,
    minibatch_size_features=16,
)



In [27]:
from sae_vis.data_storing_fns import SaeVisData


In [28]:
sae_finetune = sae_finetune.to('cuda:0')

In [29]:
sae_base = sae_base.to('cuda:0')

In [ ]:
sae_vis_data = SaeVisData.create(
    sae=sae_base,
    model=base_model,
    tokens=all_tokens[:256],
    cfg=sae_base_vis_config,
    verbose=True,
)



In [31]:
filename = "base_specific_feature_vis_random.html"
sae_vis_data.save_feature_centric_vis(filename)

In [32]:
del base_model

In [33]:
torch.cuda.empty_cache()

In [ ]:

finetuned_model = finetuned_model.to('cuda:0')

In [35]:
sae_finetune = sae_finetune.to('cuda:0')

In [36]:
del sae_base

In [ ]:
sae_vis_data = SaeVisData.create(
    sae=sae_finetune,
    model=finetuned_model,
    tokens=all_tokens[:256],
    cfg=sae_finetune_vis_config,
    verbose=True,
)



In [38]:
filename = "finetune_specific_feature_vis_random.html"
sae_vis_data.save_feature_centric_vis(filename)

Finally we can view the HTML with the latent dashboards. There is a drop down in the lop left corner to view the different latents that we specified in the config.

In [39]:
import os
import http.server
import socketserver
import threading
import IPython.display as display

PORT = 8083

def display_vis_inline(filename: str, height: int = 850):
    '''
    Displays HTML files in Jupyter Notebook. Uses global `PORT` variable defined earlier,
    so that each visualization has a unique port without having to define a port within the function.
    '''
    global PORT
    
    def serve(directory):
        os.chdir(directory)
        # Create a handler for serving files
        handler = http.server.SimpleHTTPRequestHandler
        # Create a socket server with the handler
        with socketserver.TCPServer(("", PORT), handler) as httpd:
            print(f"Serving files from {directory} on port {PORT}")
            httpd.serve_forever()
    
    # Start the server in a separate thread
    thread = threading.Thread(target=serve, args=(os.getcwd(),))
    thread.daemon = True  # This ensures the thread will be terminated when the main program exits
    thread.start()
    
    # Display the HTML file in an iframe
    iframe_html = f'<iframe src="http://localhost:{PORT}/{filename}" width="100%" height="{height}px"></iframe>'
    display.HTML(iframe_html)
    
    # Increment port for next use
    PORT += 1

In [ ]:
filename = "finetune_specific_feature_vis_random.html"
display_vis_inline(filename)